<a href="https://colab.research.google.com/github/Meguazy/HACK_4DS/blob/main/notebooks_stas/lgb_day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Meguazy/HACK_4DS.git

Cloning into 'HACK_4DS'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 65 (delta 22), reused 1 (delta 0), pack-reused 0
Receiving objects: 100% (65/65), 11.56 MiB | 4.26 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [2]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [3]:
%cd /content/HACK_4DS

/content/HACK_4DS


# Initial

In [4]:
!pip install lightgbm imbalanced-learn

In [5]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler

In [6]:
df_train = pd.read_csv("data/train_set.csv", sep=";", quotechar="\"", decimal=",")

In [7]:
df_test = pd.read_csv("data/test_set.csv", sep=";", quotechar="\"", decimal=",")

# LGB

In [8]:
df_dropped = pd.read_csv("data/dropped.csv")

In [ ]:
df_dropped.info()

In [11]:
train_columns = ['external_score_ver01',
       'external_score_ver02', 'late_payment_score',
       'external_score_late_payment_integrated', 'external_score_moderate',
       'external_score_adverse', 'external_score_ver03', 'age',
       'gross_margin_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'last_statement_age', 'overrun_freq_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'std_rel_used_a_scadenza', 'std_count_enti_affidanti',
       'max_count_numero_prima_info', 'decision_month']

In [12]:
df_dropped_all = df_dropped[train_columns]

In [13]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_dropped_all, df_dropped.target, test_size=0.2, random_state=42)

# Applying undersampling
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Creating a LightGBM dataset
train_data = lgb.Dataset(X_resampled, label=y_resampled)

# Setting up parameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05,
    'random_state': 42
}

# Training the model
num_round = 3100
bst = lgb.train(params, train_data, num_round)

# Making predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Calculating F1 score
f1 = f1_score(y_test, y_pred_binary)
print("F1 score:", f1)

[LightGBM] [Info] Number of positive: 5545, number of negative: 5545
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001607 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2366
[LightGBM] [Info] Number of data points in the train set: 11090, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
F1 score: 0.5148255051167673


# Submit solution

In [ ]:
df_test['decision_date'] = pd.to_datetime(df_test['decision_date'])

df_test['decision_month'] = df_test['decision_date'].dt.month

In [ ]:
df_test[['late_payment_score', 'external_score_late_payment_integrated', 'external_score_moderate', 'external_score_adverse', 'external_score_ver03']] = df_test[['late_payment_score', 'external_score_late_payment_integrated', 'external_score_moderate', 'external_score_adverse', 'external_score_ver03']].fillna(df_test[['late_payment_score', 'external_score_late_payment_integrated', 'external_score_moderate', 'external_score_adverse', 'external_score_ver03']].median())

In [ ]:
external_score_ver03_dictionary = {
    "MISSING":None,
    "A":14,
    "B":13,
    "C":12,
    "D":11,
    "E":10,
    "F":9,
    "G":8,
    "H":7,
    "I":6,
    "L":5,
    "M":4,
    "N":3,
    "O":2,
    "P":1
}

In [ ]:
df_test = df_test.replace({"external_score_ver03": external_score_ver03_dictionary})

In [ ]:
# Making predictions
y_test_pred = bst.predict(df_test[train_columns], num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_test_pred_binary = [1 if pred > 0.5 else 0 for pred in y_test_pred]

In [ ]:
submission = pd.DataFrame(y_test_pred_binary, columns=["label"])

In [ ]:
submission.to_csv("file.csv", index=False)